In [2]:
import collections
import itertools
import numpy as np
import pathlib
from sklearn.model_selection import StratifiedKFold

from arrays import split_dataset, load_data_txt

In [10]:
input = '/home/xandao/Imagens/pr_dataset_features/RGB/400/specific_epithet_trusted/5/vgg16/npz'
n_samples_original = 1356
n_patch =  3
n_samples_patch = n_samples_original * n_patch
n_features = 512
n_labels = 55
SEED=1234

In [11]:
x = np.empty(shape=(n_samples_patch, n_features))
y = []

for f in pathlib.Path(input).rglob('*.npz'):
    if f.is_file():
        d = np.load(f)
        np.vstack((x, d['x']))
        y.append(d['y'])

x = x.astype(np.float64)
y = list(itertools.chain(*y))
y = np.array(y, dtype=np.int16)
print(x.shape, y.shape)

(4068, 512) (4068,)


In [16]:
np.random.seed(seed=SEED)
fake_x = np.random.random_sample((n_samples_original, n_features))
fake_y = [np.repeat(k, int(v/n_patch)) for k, v in collections.Counter(y).items()]
fake_y = np.array(list(itertools.chain(*fake_y)))
print(fake_x.shape, fake_y.shape)

(1356, 512) (1356,)


In [17]:
folds = 5
kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=SEED)
index = kf.split(fake_x, fake_y)

In [18]:
def get_samples_with_patch(x, y, index_train_test, n_patch):
    new_x = np.empty(shape=(0, x.shape[1]))
    new_y = np.empty(shape=(0,))

    for index in index_train_test:
        start = (index * n_patch)
        end = start + n_patch
        new_x = np.concatenate([new_x, x[start:end]])
        new_y = np.concatenate([new_y, y[start:end]])

    return new_x, new_y


def load_a(x, y):
    for (index_train, index_test) in index:
        x_train, y_train = get_samples_with_patch(x, y, index_train, n_patch)
        x_test, y_test = get_samples_with_patch(x, y, index_test, n_patch)

In [19]:
%timeit load_a(x, y)

The slowest run took 13.41 times longer than the fastest. This could mean that an intermediate result is being cached.
396 ns ± 551 ns per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
def load_b(x, y):
    for index_train, index_test in index:
        x_train, y_train = split_dataset(index_train, n_features, n_patch, x, y)
        x_test, y_test = split_dataset(index_test, n_features, n_patch, x, y)

In [1]:
%timeit load_b(x, y)

NameError: name 'load_b' is not defined